In [2]:
import os

import mesh_to_sdf
import numpy as np
import pandas as pd
import trimesh

os.environ["PYOPENGL_PLATFORM"] = "egl"

In [3]:
def sample_points(mesh, n_points, p_type=np.float16):
    pcl, idx = mesh.sample(n_points, return_index=True)
    normals = mesh.face_normals[idx]

    out_dict = {
        "points": pcl.astype(p_type),
        "normals": normals.astype(p_type),
    }
    return out_dict


def sample_iou_points_and_sdf_vals(mesh, n_iou_points, cube_dim=32, padding=0, p_type=np.float16):
    points = (np.random.rand(n_iou_points, 3).astype(np.float32) - 0.5) * (cube_dim + padding)

    sdf_vals = mesh_to_sdf.mesh_to_sdf(mesh, query_points=points)

    occ = (sdf_vals < 0).astype(int)
    points = points.astype(p_type)

    out_dict = {
        "points": points,
        "occupancies": np.packbits(occ),
    }

    return out_dict, sdf_vals


In [15]:
os.chdir(r"/allen/aics/modeling/ritvik/projects/npm1_timeseries/mesh/")
os.getcwd()
in_mesh_path = os.listdir()[0]
in_mesh_path

'0002ea014e02cba173dd5c7210ea5df9dea7b1fcbb2d16e6fb83cf1e_10071_254.stl'

In [17]:
mesh = trimesh.load(in_mesh_path)
bbox = mesh.bounding_box.bounds
loc = (bbox[0] + bbox[1]) / 2
scale_factor = (bbox[1] - bbox[0]).max()

In [22]:
print(bbox, loc, scale_factor)
prep_mesh = mesh.apply_translation(-loc)
prep_mesh = prep_mesh.apply_scale(1 / scale_factor)
surface_data_dict = sample_points(prep_mesh, int(32**3))
surface_data_dict["loc"] = loc
surface_data_dict["scale"] = scale_factor

[[-24.19551659 -23.72639084  -3.52455902]
 [ 14.79337311  18.74357986   3.47544098]] [-4.70107174 -2.49140549 -0.02455902] 42.469970703125


In [21]:
volume_data_dict, sdf_vals = sample_iou_points_and_sdf_vals(
    prep_mesh, int(32**3), cube_dim=1
)
volume_data_dict["loc"] = loc
volume_data_dict["scale"] = scale_factor

GLError: GLError(
	err = 12290,
	baseOperation = eglInitialize,
	cArguments = (
		<OpenGL._opaque.EGLDisplay_pointer object at 0x7f3e91e148c0>,
		c_long(0),
		c_long(0),
	),
	result = 0
)

In [23]:
import pygame
from pygame.locals import *
from OpenGL.GL import *

'/allen/aics/modeling/ritvik/projects/npm1_timeseries/nuc_image'

['83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_10926_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11026_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11032_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11035_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11038_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11079_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11080_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11088_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11089_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11090_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11093_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_1113_189.tiff',
 '83bf9e5304371a9284a200dd594a457e0c4b9fa0dcf6a596c58c922f_11147_189.tiff',
 '83bf9e53043

In [ ]:
#244676 items in the mesh folder..all are .stl files

In [ ]:
r"/allen/aics/modeling/ritvik/projects/npm1_timeseries/mesh/"
path_csv = ""
out_dir = r"/allen/aics/assay-dev/users/Fatwir/benchmarking_representations/output_sdf_pc/"
df = pd.read_csv(path_csv)

for i, row in df.iterrows():
    in_mesh_path = row["mesh_path_noalign"]
    cellid = row["CellId"]
    # out_dir = f"{parent_out_dir}/{cellid}"

    mesh = trimesh.load(in_mesh_path)
    bbox = mesh.bounding_box.bounds
    loc = (bbox[0] + bbox[1]) / 2
    scale_factor = (bbox[1] - bbox[0]).max()

    prep_mesh = mesh.apply_translation(-loc)
    prep_mesh = prep_mesh.apply_scale(1 / scale_factor)

    surface_data_dict = sample_points(prep_mesh, int(32**3))
    surface_data_dict["loc"] = loc
    surface_data_dict["scale"] = scale_factor

    volume_data_dict, sdf_vals = sample_iou_points_and_sdf_vals(
        prep_mesh, int(32**3), cube_dim=1
    )
    volume_data_dict["loc"] = loc
    volume_data_dict["scale"] = scale_factor

    np.savez(f"{out_dir}/points", **volume_data_dict)
    np.savez(f"{out_dir}/pointcloud", **surface_data_dict)
    np.save(f"{out_dir}/df.npy", sdf_vals)
